In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import matplotlib as mpl
import json

import statistics

from scipy.optimize import curve_fit
from scipy import interpolate

In [ ]:
# MPL config
mpl.rc('font', size=14)

dosave = False
savedir = "/home/grayputnam/Work/Winter2023/Feb19Plots/"

In [ ]:
# Configuration

# Plane
IPLANE = 1

# Filename for WC configuration containing electron paths
WCpaths_filename = "./garfield-icarus-fnal-rev1.json"
# Time spacing in file
time = np.linspace(0,100,1000, endpoint=False)

# Filename and histogram with electronics response
fER = "elecResp_ICARUS.root"
fERh = uproot.open(fER)["ER"]

# Noise histos
fNoise = "dataFFTHistosWW.root"
fNoiseh = uproot.open(fNoise)["intpowerI1"]

# what angle to simulate
THXW_DEG = 45

# Process the configuration
dt = time[1] - time[0]

ER_val = fERh.values()
ER_time = fERh.axis().centers()
ER = interpolate.interp1d(ER_time, ER_val, kind="linear", bounds_error=False, fill_value=0)

In [ ]:
len(fNoiseh.axis().centers())

In [ ]:
plt.plot(fNoiseh.axis().centers(), fNoiseh.values())

In [ ]:
# Helper functions
def convolve(f1, f2):
    '''
    Return the simple convolution of the two arrays using FFT+mult+invFFT method.
    '''
    # fftconvolve adds an unwanted time shift
    #from scipy.signal import fftconvolve
    #return fftconvolve(field, elect, "same")
    s1 = np.fft.fft(f1)
    s2 = np.fft.fft(f2)
    sig = np.fft.ifft(s1*s2)

    return np.real(sig)

def gaus(t, sigma):
    return np.exp(-t**2/(2*sigma**2))

def agaus(t, sigmalo, sigmahi):
    return np.exp(-t**2/(2*sigmalo**2))*(t < 0) + np.exp(-t**2/(2*sigmahi**2))*(t >= 0)

def norm(v, vnorm=None):
    if vnorm is None:
        vnorm = v
    if IPLANE == 2:
        return v / vnorm.max()
    else:
        return v / np.abs(vnorm.min())
    
def center(t, v):
    if IPLANE == 0: # Ind-0 -- center on down-peak
        return t - t[np.argmin(v)]
    elif IPLANE == 2: # Collection -- center on peak
        return t - t[np.argmax(v)]
    else: # Ind-1 -- center on zero-cross
        center_ind = np.argmin(np.abs(v[np.argmax(v):np.argmin(v)])) + np.argmax(v)
        return t - t[center_ind]
    
def peak_pos(v):
    search_lo = 1160
    search_hi = 1210
    
    if IPLANE == 2:
        return np.argmax(v[search_lo:search_hi]) + search_lo
    else:
        return np.argmin(v[search_lo:search_hi]) + search_lo
    
def gen_noise(scale, noise_rand=0.1):
    noise_v = fNoiseh.values()
    noise_v = noise_v[1:len(noise_v)//2+1]
    
    r1 = np.random.random(noise_v.size)
    r2 = np.random.random(noise_v.size)
    
    pval = noise_v*((1-noise_rand) + 2*noise_rand*r1)*scale
    phase = 2*np.pi*r2
    noisevec = pval*np.exp(1j*phase)
    noisevec = np.array([0] + list(noisevec))
    return np.fft.irfft(noisevec)

N = gen_noise(1)
noise_power = np.sqrt(np.sum(N**2)/len(N)) 

noise_power

In [ ]:
dat = json.load(open(WCpaths_filename))

driftV = dat["FieldResponse"]["speed"]*1e3 # mm/us

thispaths = dat["FieldResponse"]["planes"][IPLANE]["PlaneResponse"]["paths"]

pitchpos_f = [path["PathResponse"]["pitchpos"] for path in thispaths] # mm

paths_f = [np.array(path["PathResponse"]["current"]["array"]["elements"])
             for path in thispaths]


pitchpos = []
paths = []

for i, (ppos, path) in enumerate(zip(pitchpos_f, paths_f)):
    pitchpos.append(ppos)
    paths.append(path)
    if -ppos not in pitchpos_f:
        pitchpos.append(-ppos)
        paths.append(path)
    # handle boundary between (half-)wires
    else:
        paths[-1] = tuple([paths[-1], paths_f[pitchpos_f.index(-ppos)]])
        
pitchpos, paths = zip(*sorted(zip(pitchpos, paths), key=lambda pair: pair[0]))
pitchpos = np.array(pitchpos)

In [ ]:
for p, path in zip(pitchpos, paths):
    if isinstance(path, tuple):
        plt.plot(time, -path[0])
    else:
        plt.plot(time, -path)
    
plt.title("Wire-Cell Electron Path Responses")
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

plt.xlim([58 - 2*(2-IPLANE), 66 - 2*(2-IPLANE)])
plt.tight_layout()

if dosave: plt.savefig(savedir + "pathResp.pdf")

In [ ]:
# interpolate the paths with a finer spacing (0.03mm)
# avoid edge effects by spacing in between the discontinuities in the paths (every 1.5mm)
pitchpos_interp = np.linspace(pitchpos.min(), pitchpos.max(), 2101) + 0.015
paths_interp = []

for i, p in enumerate(pitchpos_interp):
    i_pitchpos = int((p - pitchpos[0]+1e-6) / (pitchpos[1] - pitchpos[0]))
    if i_pitchpos == len(pitchpos) - 1:
        path = paths[i_pitchpos]
    else:
        F1 = paths[i_pitchpos]
        F2 = paths[i_pitchpos+1]
        
        # handle boundary between (half-)wires
        if isinstance(F1, tuple):
            if p > pitchpos[i_pitchpos]:
                F1 = F1[0]
            else:
                F1 = F1[1]
        if isinstance(F2, tuple):
            if p > pitchpos[i_pitchpos+1]:
                F2 = F2[0]
            else:
                F2 = F2[1]
                
        interp = (pitchpos[i_pitchpos+1] - p)/(pitchpos[1] - pitchpos[0])
        path = F1*interp + F2*(1-interp)

    paths_interp.append(path)

In [ ]:
# Compute the interpolated field response at this track angle
thxw = THXW_DEG*np.pi/180
shift = (np.tan(thxw)*pitchpos_interp/driftV/dt).astype(int)

shifted_paths = []
for i in range(len(paths_interp)):

    s = shift[i]
    shifted_paths.append(np.roll(paths_interp[i], s))

    if s < 0:
        shifted_paths[i][s:] = 0
    if s > 0:
        shifted_paths[i][:s] = 0
        
field_response = -sum(shifted_paths)
SR = convolve(field_response, ER(time))
timeplt = center(time, SR)

SR_power = np.max(SR)

In [ ]:
tdcs = np.linspace(-1024*0.4, 1023*0.4, 2048)

In [ ]:
# Build SR on timeticks
SR_interp = interpolate.interp1d(time, SR, kind="linear", bounds_error=False, fill_value=0)

wvf = SR_interp(tdcs)

In [ ]:
plt.plot(center(tdcs, wvf), wvf)
plt.xlim([-10, 10])

In [ ]:
plt.plot(center(tdcs, wvf), wvf + gen_noise(0.2*SR_power/noise_power))
plt.xlim([-10, 10])

In [ ]:
S2N = [10, 5, 3]

In [ ]:
allinds = []

for s2n in S2N:
    inds = []
    for _ in range(10_000):
        noise = gen_noise(SR_power/s2n/noise_power)
        ind = peak_pos(wvf+noise)
        inds.append(ind)
    
    allinds.append(inds)

In [ ]:
bins = np.linspace(-7, 7, 15)-0.5

_ = plt.hist(allinds[0] - peak_pos(wvf), 
             bins=bins, density=True, histtype="step", label="S/N: 10")
_ = plt.hist(allinds[1] - peak_pos(wvf), 
             bins=bins, density=True, histtype="step", label="S/N: 5")
_ = plt.hist(allinds[2] - peak_pos(wvf), 
             bins=bins, density=True, histtype="step", label="S/N: 3")

ticks = np.linspace(-7.5, 6.5, 201)
# plt.plot(ticks, gaus(ticks, 0.5/0.4)/np.sqrt(2*np.pi*(0.5/0.4**2)), color="black", linestyle="--")
plt.plot(ticks, agaus(ticks, 0.5/0.4, 1/0.4)/np.sqrt(2*np.pi*(0.5/0.4**2)), color="black", linestyle="--")

plt.legend()
plt.ylabel("Probability")
plt.xlabel("Selected Peak Offset")
plt.title("Plane %i, $\\theta_{xw}: %i^\\circ$" % (IPLANE, THXW_DEG))

In [ ]:
def peak_pos(v):
    search_lo = 1160
    search_hi = 1210
    
    v_search = v[search_lo:search_hi]
    v_max = np.argmax(v_search) + search_lo
    v_min = np.argmin(v_search) + search_lo
    return (v_max + v_min) / 2

In [ ]:
allinds = []

for s2n in S2N:
    inds = []
    for _ in range(10_000):
        noise = gen_noise(SR_power/s2n/noise_power)
        ind = peak_pos(wvf+noise)
        inds.append(ind)
    
    allinds.append(inds)

In [ ]:
bins = np.linspace(-7, 7, 15)-0.5

_ = plt.hist(allinds[0] - peak_pos(wvf), 
             bins=bins, density=True, histtype="step", label="S/N: 10")
_ = plt.hist(allinds[1] - peak_pos(wvf), 
             bins=bins, density=True, histtype="step", label="S/N: 5")
_ = plt.hist(allinds[2] - peak_pos(wvf), 
             bins=bins, density=True, histtype="step", label="S/N: 3")
plt.legend()
plt.ylabel("Probability")
plt.xlabel("Selected Peak Offset")
plt.title("Plane %i, $\\theta_{xw}: %i^\\circ$" % (IPLANE, THXW_DEG))